In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter, defaultdict
from sqlalchemy import create_engine
from collections import defaultdict

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [ ]:
df = pd.read_csv('cardiac_mri.csv')
df

,measurement_procedure_id,procedure_occurrence_id,visit_occurrence_id,patient_id,patient_visit_id,person_id,procedure_datetime,procedure_source_code,procedure_source_name,measurement_datetime,measurement_source_code,measurement_source_name,value_source_value,reg_dt
0,5,M0374298048200410250018,NaN,NaN,NaN,5552505776646375,2004-10-25 00:00:00.000,RM2106,Cardiac MRI - General,2004-10-26 17:11:15.000,NaN,NaN,[Finding]\n2004-10-26 CARDIAC MRI - GENERAL\...,2024-04-12
1,3,M0318727609200408040006,NaN,NaN,NaN,4956555075705976,2004-08-04 00:00:00.000,RM2103,Cardiac MRI - Congenital heart disease,2004-10-19 16:23:52.000,NaN,NaN,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...,2024-04-12
2,4,M0374204423200410250012,NaN,NaN,NaN,5552504872686170,2004-10-25 00:00:00.000,RM2103,Cardiac MRI - Congenital heart disease,2004-10-25 18:22:32.000,NaN,NaN,[Finding]\n2004-10-25 CARDIAC MRI - CONGENIT...,2024-04-12
3,6,M0374275076200409220004,NaN,NaN,NaN,5552505573646673,2004-09-22 00:00:00.000,RM2101,Cardiac MRI - Cardiomyopathy,2004-11-01 16:25:43.000,NaN,NaN,[Finding]\n2004-11-01 [추가판독]\r\n[Finding]\r\...,2024-04-12
4,7,M0372490499200410080001,NaN,NaN,NaN,5550525768686876,2004-10-08 00:00:00.000,RM2103,Cardiac MRI - Congenital heart disease,2004-11-04 15:27:56.000,NaN,NaN,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...,2024-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,5131,512180553,NaN,NaN,NaN,5448485172736476,2023-08-23 00:00:00.000,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,2023-08-23 15:55:47.000,NaN,NaN,"[Finding]\nC.I:large 2' ASD, a.fib, pulm HTN w...",2024-04-12
5290,5132,507123298,NaN,NaN,NaN,5354495573715972,2023-08-02 00:00:00.000,RM21111C,MRI Cardiac (Cine+DE+Mapping) (contrast),2023-08-23 16:09:45.000,NaN,NaN,[Finding]\nC.I>\r\ns/p ASD closure (22.02): Fi...,2024-04-12
5291,5195,501204914,NaN,NaN,NaN,5549485775676675,2023-07-07 00:00:00.000,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,2023-10-06 14:37:29.000,NaN,NaN,[Finding]\nC.I.> #TOF with PA\r\nCompared with...,2024-04-12
5292,5196,481297907,NaN,NaN,NaN,5156485576706272,2023-04-14 00:00:00.000,RM2131C,MRI Cardiac (Cine+DE) (contrast),2023-10-06 15:01:16.000,NaN,NaN,[Finding]\nCI:Septal hypertrophy\r\n\r\n1. LV ...,2024-04-12


In [ ]:
df = df[['measurement_datetime','procedure_source_code','procedure_source_name','value_source_value']].sort_values('measurement_datetime', ascending=True, ignore_index=True)
df

,measurement_datetime,procedure_source_code,procedure_source_name,value_source_value
0,2004-10-19 16:23:52.000,RM2103,Cardiac MRI - Congenital heart disease,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...
1,2004-10-25 18:22:32.000,RM2103,Cardiac MRI - Congenital heart disease,[Finding]\n2004-10-25 CARDIAC MRI - CONGENIT...
2,2004-10-26 17:11:15.000,RM2106,Cardiac MRI - General,[Finding]\n2004-10-26 CARDIAC MRI - GENERAL\...
3,2004-11-01 16:25:43.000,RM2101,Cardiac MRI - Cardiomyopathy,[Finding]\n2004-11-01 [추가판독]\r\n[Finding]\r\...
4,2004-11-04 15:27:56.000,RM2103,Cardiac MRI - Congenital heart disease,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...
...,...,...,...,...
5289,2023-12-28 17:13:41.000,RM2131C,MRI Cardiac (Cine+DE) (contrast),"[Finding]\nC.I> CABG 1yr f/u, compared with 20..."
5290,2023-12-29 14:02:44.000,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,"[Finding]\nC.I. : Progressed PS\r\n# TOF, LPA ..."
5291,2023-12-29 15:04:24.000,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,"[Finding]\nC.I.> f/u \r\n# ccTGA, TR, IVC inte..."
5292,2023-12-29 15:55:43.000,RM2131C,MRI Cardiac (Cine+DE) (contrast),[Finding]\nCI > CABG 1yr f/u\r\n# postinfarct ...


In [ ]:
df[['procedure_source_code','procedure_source_name']].value_counts()

procedure_source_code  procedure_source_name                                   
RM2121C                MRI Cardiac + CINE - Congenital heart disease (contrast)    1970
RM2111C                MRI Cardiac + CINE - Cardiomyopathy (contrast)              1334
RM21211C               MRI Cardiac - Congenital (Cine+VENC+DE) (contrast)           858
RM21111C               MRI Cardiac (Cine+DE+Mapping) (contrast)                     559
RM2131C                MRI Cardiac (Cine+DE) (contrast)                             322
                       MRI Cardiac + CINE - General (contrast)                      148
RM21211                MRI Cardiac - Congenital (Cine+VENC) (noncontrast)            87
RM2103                 Cardiac MRI - Congenital heart disease                         9
RM2106                 Cardiac MRI - General                                          4
RM2101                 Cardiac MRI - Cardiomyopathy                                   3
Name: count, dtype: int64

In [ ]:
text = df[df['procedure_source_code']=='RM2121C'].iloc[0, -1]
print(text)

[Finding]
2005-01-18   


[Conclusion]
1. severe PR
(regurgitation fraction= MPA reverse flow/MPA forward flow =54.26/112.03=0.48)
2. RVE but relatively conserved RV systolic function.
3. no TR, no PA stenosis.

                forward flow    reverse flow        net forward flow   (Normalized)
Aorta         73.19                 0                         73.19 (59.61)
MPA           112.03              54.26                  57.77 (47.06)
RPA           62.35                30.24                  32.11 (26.16)
LPA           39.29                16.95                   22.34 (18.19)


LV  EF      61.28%                           RV   EF    41.31%
       EDV   105.80ml (86.18)               EDV  239.83ml (195.35)  
       ESV   40.97ml (33.37)                     ESV   140.75ml (114.62)
       SV      64.83ml (52.81)                     SV     99.07ml (80.70)
       CO      5.89L/min (4.80)                  CO     9.01L/min (7.34)

BSA = 1.23m^2


In [ ]:
# Define regular expressions to capture the required values
patterns = {
    "Delayed Enhancement": re.compile(r"Delayed enhancement", re.DOTALL | re.IGNORECASE),

    "RV EF": re.compile(r"RV.*?Ejection Fraction\s*:\s*([\d\.]+)\s*%", re.DOTALL | re.IGNORECASE),
    "RV EDV": re.compile(r"RV.*?End-Diastolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV ESV": re.compile(r"RV.*?End-Systolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV SV": re.compile(r"RV.*?Stroke Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV CO": re.compile(r"RV.*?Cardiac Output\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*l/min", re.DOTALL | re.IGNORECASE),
    "RV MM": re.compile(r"RV.*?Average Myocardial Mass\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*g", re.DOTALL | re.IGNORECASE),

    "LV EF": re.compile(r"LV.*?Ejection Fraction\s*:\s*([\d\.]+)\s*%", re.DOTALL),
    "LV EDV": re.compile(r"LV.*?End-Diastolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV ESV": re.compile(r"LV.*?End-Systolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV SV": re.compile(r"LV.*?Stroke Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV CO": re.compile(r"LV.*?Cardiac Output\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*l/min", re.DOTALL | re.IGNORECASE),
    "LV MM": re.compile(r"LV.*?Average Myocardial Mass\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*g", re.DOTALL | re.IGNORECASE),
}

# Extract the values using the regular expressions
all_results = []
results = {'ID': 1}
for key, pattern in patterns.items():
    matches = pattern.findall(text)
    for match in matches:
        print(match, key)
        if 'Delayed Enhancement' in key:
            results[key] = "Yes"
        elif 'EF' in key:
            results[key] = match
        else:
            results[key+'(Value)'] = match[0]
            results[key+'(Index)'] = match[1]
all_results.append(results)

# Create a DataFrame from the extracted results
all_results_df = pd.DataFrame(all_results)
all_results_df

,ID
0,1


In [ ]:
text = df[df['procedure_source_code']=='RM2121C'].iloc[200, -1]
print(text)

[Finding]
2009-10-06   


[Conclusion]
1. s/p sternotomy. (no information about surgery is available)
    I.D.D type ccTGA
    Mesocardia, left aortic arch

    Global hypokinesia of RV 
    Delayed enhancement at RV mid inferior wall. Focal high signal at apex and papillary m.

    Hypertrophic RV and thinning of LV wall
    RAE
    Severe MR (fraction : 54%)
    Small pulmonary annulus
    Supravalvar pulmonary stenosis (18mm) or pulmonary banding state
    Post stenotic dilatation of MPA

2. LV Function Measurement <absolute (normalized)>
 - Ejection Fraction : 46.4 %
 - End-Diastolic Volume : 241.8 (147.2) ml
 - End-Systolic Volume : 129.7 (79.0) ml
 - Stroke Volume : 112.1 (68.3) ml
 - Cardiac Output : 7.85 (4.78) L/min
 - Average Myocardial Mass : 62.3 (37.9) g
 - BSA : 1.64 m2
 - Heart Rate : 70 bpm

3. RV (systemic) Function Measurement <absolute (normalized)>
 - Ejection Fraction : 33.5 %
 - End-Diastolic Volume : 199.4 (121.4) ml
 - End-Systolic Volume : 132.6 (80.7) ml
 - St

In [ ]:
# Define regular expressions to capture the required values
patterns = {
    "EF": re.compile(r'EF\s+(\d+\.\d+)%'),
    "EDV": re.compile(r'EDV\s+(\d+\.\d+)ml\s*\(([\d\.]+)\)'),
    "ESV": re.compile(r'ESV\s+(\d+\.\d+)ml\s*\(([\d\.]+)\)'),
    "SV": re.compile(r'(?<!E)SV\s+(\d+\.\d+)ml\s*\(([\d\.]+)\)'),
    "CO": re.compile(r'CO\s+(\d+\.\d+)L/min\s*\(([\d\.]+)\)')
}

# Extract the values using the regular expressions
all_results = []
results = {'ID': 1}
for key, pattern in patterns.items():
    matches = pattern.findall(text)
    print(matches, key)
    if key == 'EF':
        results['LV EF'] = matches[0]
        results['RV EF'] = matches[1]
    else:
        results[f'LV {key}(Value)'] = matches[0][0]
        results[f'LV {key}(Index)'] = matches[0][1]
        results[f'RV {key}(Value)'] = matches[1][0]
        results[f'RV {key}(Index)'] = matches[1][1]
all_results.append(results)

# Create a DataFrame from the extracted results
all_results_df = pd.DataFrame(all_results)
all_results_df

[] EF


IndexError: list index out of range

In [ ]:
########## 전체 데이터
# Define regular expressions to capture the required values
patterns = {
    "Delayed Enhancement": re.compile(r"Delayed enhancement", re.DOTALL | re.IGNORECASE),
    "No Delayed Enhancement": re.compile(r"No.*?Delayed enhancement", re.IGNORECASE),

    "RV EF": re.compile(r"RV.*?Ejection Fraction\s*:\s*([\d\.]+)\s*%", re.DOTALL | re.IGNORECASE),
    "RV EDV": re.compile(r"RV.*?End-Diastolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV ESV": re.compile(r"RV.*?End-Systolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV SV": re.compile(r"RV.*?Stroke Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV CO": re.compile(r"RV.*?Cardiac Output\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*l/min", re.DOTALL | re.IGNORECASE),
    "RV MM": re.compile(r"RV.*?Average Myocardial Mass\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*g", re.DOTALL | re.IGNORECASE),

    "LV EF": re.compile(r"LV.*?Ejection Fraction\s*:\s*([\d\.]+)\s*%", re.DOTALL),
    "LV EDV": re.compile(r"LV.*?End-Diastolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV ESV": re.compile(r"LV.*?End-Systolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV SV": re.compile(r"LV.*?Stroke Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV CO": re.compile(r"LV.*?Cardiac Output\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*l/min", re.DOTALL | re.IGNORECASE),
    "LV MM": re.compile(r"LV.*?Average Myocardial Mass\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*g", re.DOTALL | re.IGNORECASE),
}

# fallback_patterns = {
#     "EF": re.compile(r'EF\s+(\d+\.\d+)%'),
#     "EDV": re.compile(r'EDV\s+(\d+\.\d+)ml\s*\(([\d\.]+)\)'),
#     "ESV": re.compile(r'ESV\s+(\d+\.\d+)ml\s*\(([\d\.]+)\)'),
#     "SV": re.compile(r'(?<!E)SV\s+(\d+\.\d+)ml\s*\(([\d\.]+)\)'),
#     "CO": re.compile(r'CO\s+(\d+\.\d+)L/min\s*\(([\d\.]+)\)')
# }

columns = ['Date', 'Code', 'Text', 'Delayed Enhancement',
           'LV EF', 'LV EDV(Value)', 'LV EDV(Index)', 'LV ESV(Value)', 'LV ESV(Index)',
           'LV SV(Value)', 'LV SV(Index)', 'LV CO(Value)', 'LV CO(Index)', 'LV MM(Value)', 'LV MM(Index)',
           'RV EF', 'RV EDV(Value)', 'RV EDV(Index)', 'RV ESV(Value)', 'RV ESV(Index)',
           'RV SV(Value)', 'RV SV(Index)', 'RV CO(Value)', 'RV CO(Index)', 'RV MM(Value)', 'RV MM(Index)']

all_results = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['value_source_value']
    results = {'Date': row['measurement_datetime'], 'Code': row['procedure_source_code'], 'Text':text}

    # for key, pattern in fallback_patterns.items():
    #     matches = pattern.findall(text)
    #     print(matches, key)
    #     if matches:
    #         if key == 'EF':
    #             results['LV EF'] = matches[0]
    #             results['RV EF'] = matches[1]
    #         else:
    #             results[f'LV {key}(Value)'] = matches[0][0]
    #             results[f'LV {key}(Index)'] = matches[0][1]
    #             results[f'RV {key}(Value)'] = matches[1][0]
    #             results[f'RV {key}(Index)'] = matches[1][1]

    for key, pattern in patterns.items():
        if (key in results) or (key+'(Value)' in results) or (key+'(Index)' in results): # results에 이미 key 값이 있으면 continue
            continue
        matches = pattern.findall(text)
        for match in matches:
            if 'Delayed Enhancement' in key:
                results[key] = "Yes"
                if 'No Delayed Enhancement' in key:
                    results['Delayed Enhancement'] = "No"
            elif 'EF' in key:
                results[key] = match
            else:
                results[key+'(Value)'] = match[0]
                results[key+'(Index)'] = match[1]


    all_results.append(results)

# Create a DataFrame from the extracted results
all_results_df = pd.DataFrame(all_results)
all_results_df = all_results_df[columns]
all_results_df

  0%|          | 0/5294 [00:00<?, ?it/s]

,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
0,2004-10-19 16:23:52.000,RM2103,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-10-25 18:22:32.000,RM2103,[Finding]\n2004-10-25 CARDIAC MRI - CONGENIT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-10-26 17:11:15.000,RM2106,[Finding]\n2004-10-26 CARDIAC MRI - GENERAL\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-11-01 16:25:43.000,RM2101,[Finding]\n2004-11-01 [추가판독]\r\n[Finding]\r\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-11-04 15:27:56.000,RM2103,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,2023-12-28 17:13:41.000,RM2131C,"[Finding]\nC.I> CABG 1yr f/u, compared with 20...",Yes,51.4,97.9,63.1,47.6,30.7,50.3,32.4,3.57,2.30,122.7,79.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5290,2023-12-29 14:02:44.000,RM21211C,"[Finding]\nC.I. : Progressed PS\r\n# TOF, LPA ...",Yes,54.2,122.0,81.3,55.9,37.3,66.1,44.1,5.02,3.35,60.6,40.4,46.5,179.9,119.9,96.2,64.1,83.7,55.8,6.36,4.24,60.6,40.4
5291,2023-12-29 15:04:24.000,RM21211C,"[Finding]\nC.I.> f/u \r\n# ccTGA, TR, IVC inte...",Yes,68.6,102.9,73.5,32.3,23.1,70.6,50.4,5.01,3.58,NaN,NaN,46.5,157.3,112.4,84.1,60.1,73.2,52.3,5.20,3.71,77.3,55.2
5292,2023-12-29 15:55:43.000,RM2131C,[Finding]\nCI > CABG 1yr f/u\r\n# postinfarct ...,Yes,56.8,102.8,68.1,44.4,29.4,58.4,38.7,4.96,3.29,112.9,74.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_results_df[all_results_df['RV EF'].notna()]

,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
65,2005-10-25 15:54:26.000,RM2111C,[Finding]\n2005-10-25 \n\n\n[Conclusion]\n1....,NaN,NaN,NaN,NaN,NaN,NaN,64.8,38.2,5.06,2.98,NaN,NaN,53,NaN,NaN,NaN,NaN,64.8,38.2,5.06,2.98,NaN,NaN
66,2005-10-26 16:42:11.000,RM2111C,[Finding]\n2005-10-26 \n\n\n[Conclusion]\n1....,NaN,NaN,NaN,NaN,NaN,NaN,105.9,60.0,5.93,3.41,NaN,NaN,81.6,NaN,NaN,NaN,NaN,105.9,60.0,5.93,3.41,NaN,NaN
94,2006-01-12 16:00:13.000,RM2121C,[Finding]\n2006-01-12 \n\n\n[Conclusion]\n1....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,2006-02-06 15:39:11.000,RM2131C,[Finding]\n2006-02-06 \n\n\n[Conclusion]\n1....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,2006-02-20 16:39:53.000,RM2131C,[Finding]\n2006-02-20 LV Data\r\n1> Ejection...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5286,2023-12-27 14:21:41.000,RM21211C,"[Finding]\nC.I. > # DORV, non-committed VSD, P...",Yes,68.6,108.2,67.6,34.0,21.3,74.2,46.4,5.86,3.66,71.3,44.6,47.4,204.0,127.5,107.4,67.1,96.6,60.4,7.63,4.77,71.3,44.6
5287,2023-12-27 15:06:18.000,RM21211C,[Finding]\nC.I. > TOF 후 PPVR 대상자입니다. MAIN PA 가...,NaN,63.5,119.4,66.3,43.6,24.2,75.8,42.1,9.02,5.01,96.0,53.3,30.3,243.5,135.3,169.7,94.3,73.8,41.0,8.78,4.88,96.0,53.3
5288,2023-12-27 15:38:56.000,RM2131C,[Finding]\nC.I> HCM\r\n\r\n1.5T Cardiac MRI\r\...,Yes,76.0,127.9,70.3,NaN,NaN,97.3,53.4,6.7,3.7,190.7,104.7,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5290,2023-12-29 14:02:44.000,RM21211C,"[Finding]\nC.I. : Progressed PS\r\n# TOF, LPA ...",Yes,54.2,122.0,81.3,55.9,37.3,66.1,44.1,5.02,3.35,60.6,40.4,46.5,179.9,119.9,96.2,64.1,83.7,55.8,6.36,4.24,60.6,40.4


In [ ]:
all_results_df[all_results_df['Delayed Enhancement']=='No']

,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
6,2004-11-18 16:15:43.000,RM2101,[Finding]\n2004-11-18 CARDIAC MRI - CARDIOMY...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2005-01-24 16:14:42.000,RM2111C,[Finding]\n2005-01-24 \n\n\n[Conclusion]\n1....,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2005-05-27 15:41:36.000,RM2111C,[Finding]\n2005-05-27 \n\n\n[Conclusion]\nse...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2005-06-03 16:34:32.000,RM2111C,[Finding]\n2005-06-03 \n\n\n[Conclusion]\nC....,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2005-08-02 15:29:43.000,RM2111C,[Finding]\n2005-08-02 \n\n\n[Conclusion]\nC....,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5250,2023-11-24 15:20:51.000,RM21211C,[Finding]\n# PA with IVS \r\n s/p 2014/7/31 pu...,No,62.9,63.9,63.9,23.7,23.7,40.2,40.2,3.46,3.46,31.7,31.7,60.3,59.9,59.9,23.8,23.8,36.1,36.1,3.10,3.10,NaN,NaN
5258,2023-11-30 14:57:19.000,RM21211C,[Finding]\nC.I.> Hypoplastic right heart syndr...,No,66.5,54.9,78.4,18.4,26.3,36.5,52.1,3.61,5.16,23.8,34.0,58.6,72.5,103.6,30.0,42.9,42.5,60.7,4.38,6.25,23.8,34.0
5262,2023-12-04 15:57:17.000,RM21211C,[Finding]\nCI:PA with IVS_Ebstein anomaly\r\ns...,No,55.8,55.4,79.1,24.5,35.0,30.9,44.1,2.97,4.24,81.6,116.6,65.0,31.1,44.4,10.9,15.6,20.2,28.9,1.94,2.77,81.6,116.6
5272,2023-12-12 14:51:35.000,RM21211C,[Finding]\nCI: f/u\r\n# PA IVS\r\n - s/p RV ov...,No,60.0,65.7,93.9,26.3,37.6,39.4,56.3,2.56,3.66,31.3,44.7,51.2,115.4,164.9,56.3,80.4,59.1,84.4,3.84,5.49,31.3,44.7


In [ ]:
all_results_df['Delayed Enhancement'] = all_results_df['Delayed Enhancement'].fillna('No')
all_results_df[all_results_df['Delayed Enhancement']=='No']

,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
0,2004-10-19 16:23:52.000,RM2103,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-10-25 18:22:32.000,RM2103,[Finding]\n2004-10-25 CARDIAC MRI - CONGENIT...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-10-26 17:11:15.000,RM2106,[Finding]\n2004-10-26 CARDIAC MRI - GENERAL\...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-11-01 16:25:43.000,RM2101,[Finding]\n2004-11-01 [추가판독]\r\n[Finding]\r\...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-11-04 15:27:56.000,RM2103,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5278,2023-12-18 16:20:40.000,RM21211,"[Finding]\nC.I. > RV volume 및 RVEF, PR 정도 등에 대...",No,51.3,116.1,75.9,56.6,37.0,59.5,38.9,4.40,2.88,74.5,48.7,43.4,167.9,109.4,95.0,61.9,72.9,47.5,5.40,3.52,74.5,48.7
5279,2023-12-19 14:55:18.000,RM21211C,"[Finding]\nC.I. > S/P PVR, RVOT patch widening...",No,71.8,150.4,71.6,42.4,20.2,108.0,51.4,8.21,3.91,108.5,51.7,38.4,253.6,120.8,156.1,74.3,97.5,46.4,7.41,3.53,108.5,51.7
5280,2023-12-19 14:58:44.000,RM21211,[Finding]\nCI:Mild PS with severe PR\r\n\r\nRV...,No,31.5,132.4,67.5,90.7,46.3,41.7,21.3,4.13,2.11,165.7,84.5,29.5,376.2,191.9,265.3,135.4,110.9,56.6,10.97,5.60,165.7,84.5
5283,2023-12-22 16:31:43.000,RM21111C,[Finding]\nC.I> HCM f/u\r\n\r\n3T Cardiac MRI\...,No,55.6,135.3,82.5,60.1,36.6,75.2,45.9,3.84,2.34,208.1,126.9,55.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_results_df.dropna()

,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
149,2006-10-18 19:20:17.000,RM2111C,[Finding]\n2006-10-18 \n\n\n[Conclusion]\n0....,No,62.4,148.6,72.0,55.8,27.0,92.8,45.0,6.18,2.99,115.7,56.1,44.8,109.4,53.0,60.4,29.3,49.0,23.7,3.26,1.58,105.6,51.2
182,2007-07-04 16:19:54.000,RM2121C,[Finding]\n2007-07-04 \n\n\n[Conclusion]\n1....,Yes,40.2,188.7,114.1,112.9,68.3,75.8,45.8,4.70,2.84,179.8,108.8,20.8,119.4,72.2,94.5,57.2,24.9,15.0,1.54,1.54,97.3,58.9
183,2007-07-05 16:28:32.000,RM2131C,[Finding]\n2007-07-05 \n\n\n[Conclusion]\n1....,No,53.7,137.5,88.0,63.7,40.7,73.9,47.2,5.17,3.31,55.8,35.7,47.0,196.4,125.6,104.1,66.6,92.3,59.0,6.46,4.13,55.8,35.7
184,2007-07-26 16:36:53.000,RM2121C,[Finding]\n2007-07-26 \n\n\n[Conclusion]\n1....,No,63.5,63.6,75.6,23.2,27.6,40.4,48.0,2.99,3.56,29.4,35.0,38.2,48.6,57.8,30.1,35.8,18.6,22.1,1.37,1.63,29.4,35.0
185,2007-07-30 15:24:10.000,RM2131C,[Finding]\n2007-07-30 \n\n\n[Conclusion]\n1....,No,59.0,146.6,81.6,60.1,33.5,86.5,48.2,7.18,4.00,103.2,57.4,34.0,259.1,144.2,171.1,95.2,88.0,49.0,7.30,4.07,103.2,57.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,2023-12-19 14:55:18.000,RM21211C,"[Finding]\nC.I. > S/P PVR, RVOT patch widening...",No,71.8,150.4,71.6,42.4,20.2,108.0,51.4,8.21,3.91,108.5,51.7,38.4,253.6,120.8,156.1,74.3,97.5,46.4,7.41,3.53,108.5,51.7
5280,2023-12-19 14:58:44.000,RM21211,[Finding]\nCI:Mild PS with severe PR\r\n\r\nRV...,No,31.5,132.4,67.5,90.7,46.3,41.7,21.3,4.13,2.11,165.7,84.5,29.5,376.2,191.9,265.3,135.4,110.9,56.6,10.97,5.60,165.7,84.5
5286,2023-12-27 14:21:41.000,RM21211C,"[Finding]\nC.I. > # DORV, non-committed VSD, P...",Yes,68.6,108.2,67.6,34.0,21.3,74.2,46.4,5.86,3.66,71.3,44.6,47.4,204.0,127.5,107.4,67.1,96.6,60.4,7.63,4.77,71.3,44.6
5287,2023-12-27 15:06:18.000,RM21211C,[Finding]\nC.I. > TOF 후 PPVR 대상자입니다. MAIN PA 가...,No,63.5,119.4,66.3,43.6,24.2,75.8,42.1,9.02,5.01,96.0,53.3,30.3,243.5,135.3,169.7,94.3,73.8,41.0,8.78,4.88,96.0,53.3


In [ ]:
all_results_df[all_results_df['RV EF'].notna()].sample(1000).to_csv('MRI_Regex_1000.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 아무런 항목이 추출되지 않은 경우 확인

columns_to_check = [
    'LV EF', 'LV EDV(Value)',
    'LV EDV(Index)', 'LV SV(Value)', 'LV SV(Index)', 'LV CO(Value)',
    'LV CO(Index)', 'LV MM(Value)', 'LV MM(Index)', 'RV EF',
    'RV EDV(Value)', 'RV EDV(Index)', 'RV SV(Value)', 'RV SV(Index)',
    'RV CO(Value)', 'RV CO(Index)', 'RV MM(Value)', 'RV MM(Index)'
]

all_results_df[all_results_df[columns_to_check].isnull().all(axis=1)]

,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
0,2004-10-19 16:23:52.000,RM2103,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-10-25 18:22:32.000,RM2103,[Finding]\n2004-10-25 CARDIAC MRI - CONGENIT...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-10-26 17:11:15.000,RM2106,[Finding]\n2004-10-26 CARDIAC MRI - GENERAL\...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-11-01 16:25:43.000,RM2101,[Finding]\n2004-11-01 [추가판독]\r\n[Finding]\r\...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-11-04 15:27:56.000,RM2103,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5009,2023-05-24 14:33:12.000,RM21111C,[Finding]\n\n \n\n[Conclusion]\n* Addendum Rep...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5026,2023-06-05 16:37:21.000,RM21211C,[Finding]\n\n \n\n[Conclusion]\nNo significant...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5069,2023-07-04 15:34:10.000,RM21111C,[Finding]\nC.I.>\r\n# r/o Takayasu's arteritis...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5154,2023-09-08 16:24:40.000,RM21211C,[Finding]\nC.I.> R/O chemo-induced cardiomyopa...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
text = all_results_df[all_results_df['RV EF'].isnull()].iloc[10, 2]
print(text)

[Finding]
2004-12-17   1.  PR
2. regurgitation fraction
  --> (on ventricular volumetry) 20/70 = 29%
  --> (on VENC study) 34/92 = 37%

3. BSA = 1.25m^2

                forward flow    reverse flow        net forward flow  (Normalized)

Aorta       58.7                 0                          58.7(46.98)
MPA       91.79                42.88                  48.9(39.14)
RPA        60.38               21.14                  39.23(31.40)
LPA         32.42               13.28                  19.14(15.32)
TV            52.08               0.81                    51.27(41.03)


RV  EF     30.3%                            LV   EF      55.53%
      EDV   234.69(187.79) ml                EDV  95.42(76.35)  
      ESV   164.2ml(131.4)                      ESV  42.43(33.96)
      SV      70.47(56.39)ml                      SV     52.98(42.4)
      CO      5.56 L/min(4.45)                   CO     4.18(3.35)


[Conclusion]
severe PR
regurgitation fraction= MPA reverse flow/RV EDV =106/135.70=

In [ ]:
text = '''
[Finding]
C.I.: S/P TOF total correction.

1. LV Function Measurement <absolute (normalized)>
 - Ejection Fraction : 54.1 %
 - End-Diastolic Volume : 210.6 (102.9) ml
 - End-Systolic Volume : 96.7 (47.3) ml
 - Stroke Volume : 113.9 (55.7) ml
 - Cardiac Output : 7.63 (0.00) l/min
 - Average Myocardial Mass : 136.0 (66.5) g

 - BSA : 2.05 m2
 - Heart Rate : 67 bpm

2. RV Function Measurement <absolute (normalized)>
 - Ejection Fraction : 34.1 %
 - End-Diastolic Volume : 334.8 (163.6) ml
 - End-Systolic Volume : 220.7 (107.8) ml
 - Stroke Volume : 114.1 (55.8) ml
 - Cardiac Output : 7.64 (0.00) l/min

3. Forward Vol. Reverse Vol.  Net Forward Vol.
AAorta 109.18  0.87   108.31
DAorta 77.22  4.67   72.55
SVC 36.37  0.07   36.30
dTA+SVC 113.59  4.74   108.85
IVC 87.65  3.54   84.11
SVC+IVC 124.02  3.61   120.41
MPA 177.54  43.50   134.04
RPA 122.83  31.22   91.61
LPA 31.02  13.66   17.36
RPA+LPA 153.85  44.88   108.97


[Conclusion]
1. Dilated RV cavity with decreased RV systolic function.
 - Ejection Fraction : 34.1 % (<- 49.5%)
 - End-Diastolic Volume : 334.8 (163.6) ml (<- 316.6 (164.6))

2. Preserved LV systolic function with normal cavity size.
  - Ejection Fraction : 54.1 % (<- 49.6 %)
  - End-Diastolic Volume : 210.6 (102.9) ml (<-178.7 (92.9))

3. Moderate PR.
Regurgitant fraction
MPA 기준 24.50% (<-28.6%)
LPA+RPA 기준 29.17% (<-26.1%)

4. Mild stenosis of Distal MPA: 1.5 cm
Less prominent stenosis of Prox. LPA (13mm<-10 mm) with angulation.

Distal RPA: 21 mm.
Mild stenosis of Prox. RPA (14 mm).
Distal LPA: 20 mm.

5. Lung perfusion.
Rt. : Lt. = 84.0% : 16.0%

6. No RWMA.
7. RV inferior attachment site delayed enhancement
'''
results = {'Date': row['measurement_datetime'], 'Code': row['procedure_source_code'], 'Text':text}

for key, pattern in patterns.items():
    matches = pattern.findall(text)
    for match in matches:
        if (key in results) or (key+'(Value)' in results) or (key+'(Index)' in results): # results에 이미 key 값이 있으면 continue
            continue
        print(key, matches)
        if 'Delayed Enhancement' in key:
            results[key] = "Yes"
            if 'No Delayed Enhancement' in key:
                results['Delayed Enhancement'] = "No"
        elif 'EF' in key:
            results[key] = match
        else:
            results[key+'(Value)'] = match[0]
            results[key+'(Index)'] = match[1]

results

Delayed Enhancement ['delayed enhancement']
RV EF ['34.1', '34.1', '54.1']
RV EDV [('334.8', '163.6'), ('334.8', '163.6'), ('210.6', '102.9')]
RV ESV [('220.7', '107.8')]
RV SV [('114.1', '55.8')]
RV CO [('7.64', '0.00')]
LV EF ['54.1', '54.1']
LV EDV [('210.6', '102.9'), ('210.6', '102.9')]
LV ESV [('96.7', '47.3')]
LV SV [('113.9', '55.7')]
LV CO [('7.63', '0.00')]
LV MM [('136.0', '66.5')]


{'Date': '2023-12-29 16:36:50.000',
 'Code': 'RM2131C',
 'Text': '\n[Finding]\nC.I.: S/P TOF total correction.\n\n1. LV Function Measurement <absolute (normalized)>    \n - Ejection Fraction : 54.1 %    \n - End-Diastolic Volume : 210.6 (102.9) ml    \n - End-Systolic Volume : 96.7 (47.3) ml    \n - Stroke Volume : 113.9 (55.7) ml    \n - Cardiac Output : 7.63 (0.00) l/min    \n - Average Myocardial Mass : 136.0 (66.5) g    \n    \n - BSA : 2.05 m2    \n - Heart Rate : 67 bpm    \n    \n2. RV Function Measurement <absolute (normalized)>    \n - Ejection Fraction : 34.1 %    \n - End-Diastolic Volume : 334.8 (163.6) ml    \n - End-Systolic Volume : 220.7 (107.8) ml    \n - Stroke Volume : 114.1 (55.8) ml    \n - Cardiac Output : 7.64 (0.00) l/min    \n    \n3. Forward Vol. Reverse Vol.  Net Forward Vol.\nAAorta 109.18  0.87   108.31 \nDAorta 77.22  4.67   72.55 \nSVC 36.37  0.07   36.30 \ndTA+SVC 113.59  4.74   108.85 \nIVC 87.65  3.54   84.11 \nSVC+IVC 124.02  3.61   120.41 \nMPA 177.5